# マルチタスク強化学習の種類と解き方など

マルチタスク強化学習にもいろいろな問題設定の種類があり，解き方も様々です．いくつかまとめてみました．
全体を把握するのには次の論文が参考になるかも
* [When Is Generalizable Reinforcement Learning Tractable?](https://arxiv.org/abs/2101.00300)

表記
* 基本的にMDPの集合を$\mathcal{M}$と書きます．
* エピソードの数は$K$とします．

## 1. 全てのMDPの学習コストを減らしたい（Average Performance Setting）

論文：
* [Provably Efficient Multi-Task Reinforcement Learning with Model Transfer](https://arxiv.org/abs/2107.08622)：
  * リグレットの解析．
  * 学習中に複数のタスクで情報を交換する．
  * 独立して学習するよりマシなリグレットに．
* [Sample Complexity of Multi-task Reinforcement Learning](https://arxiv.org/abs/1309.6821)：
  * サンプル効率の解析だが，実質リグレットっぽい？読みづらい．
  * どのタスクを解いているかはわからない設定．
* [On the Power of Pre-training for Generalization in RL: Provable Benefits and Hardness](https://arxiv.org/abs/2210.10464)：
  * ちゃんと読もう．
  * 期待値の意味で最適な方策を見つけるアルゴリズムを提案してる：$\pi^*(\mathbb{D})=\arg \max _{\pi \in \Pi} \mathbb{E}_{\mathcal{M} \sim \mathbb{D}} V_{\mathcal{M}}^\pi$
  * 新しいタスクで探索が可能な場合とそうでない場合について解析してる．探索が不可能でも，期待値の意味で$\varepsilon$-最適な方策は見つかる

**基本的なアイディア**

MDPの集合$\mathcal{M}$の中のMDPは似通っている（例えば遷移確率の距離が近い，など）．
共通する知識を利用することで，全体のMDPを独立して学習させるよりも効率の良い学習を実現する．

**評価方法**

$$
\operatorname{Reg}(K)=\sum_{m=1}^{|\mathcal{M}|} \sum_{k=1}^K\left(V_{m}^{\star}-V_{m}^{\pi^k(m)}\right)
$$

$$\max _\pi \mathbb{E}_{M \sim \mathcal{D}}\left[V_M^{s_0}(\pi)\right]$$

## 2. 新しいタスクでの学習コストを減らしたい（Meta RL Setting）

論文：

* [On the Power of Pre-training for Generalization in RL: Provable Benefits and Hardness](https://arxiv.org/abs/2210.10464)
* [Provable Benefit of Multitask Representation Learning in Reinforcement Learning](https://arxiv.org/abs/2206.05900)
* [On the Power of Multitask Representation Learning in Linear MDP](https://arxiv.org/abs/2106.08053)
* [Provable Representation Learning for Imitation Learning via Bi-level Optimization](https://arxiv.org/abs/2002.10544)

**基本的なアイディア**

事前学習によって，新しく出会うMDP $m^*\in \mathcal{M}$で学習する手間を減らしたい．

**評価方法**

$$
\max _\pi V_{\mathrm{M}_{\text {test }}}^{s_0}(\pi)
$$


## 3. 何のタスクを解いているか推定する（Task identification？）

論文：
* [RL for Latent MDPs: Regret Guarantees and a Lower Bound](https://arxiv.org/abs/2102.04939)
* [Contextual Markov Decision Processes](https://arxiv.org/abs/1502.02259)
* [Understanding Domain Randomization for Sim-to-real Transfer](https://arxiv.org/abs/2110.03239)

**基本的なアイディア**

目的は１に近いがアプローチが若干異なる？探索時に何のタスクを解いているかが不明なので，今解いているタスクの文脈を推定するアプローチ．




参考：
* [Sample Complexity of Multi-task Reinforcement Learning](https://arxiv.org/abs/1309.6821)：もっと良い論文ありそう．問題設定と解き方は参考になる．

強化学習でマルチタスクを扱う場合，しばしば転移学習が用いられますが，なぜそれが良いのか？は自明ではありません．
強化学習におけるサンプル効率の理論を見てみましょう．

## マルチタスク強化学習アルゴリズム

次の手順でマルチタスクMDPを解きます

1. 入力：サンプルするタスクの数$T$と分類するMDPの数$\bar{C}$
2. for $t=1,2, \ldots, T_1$ do
    1. $M_t \in \mathcal{M}$をサンプル
    2. $M_t$において，$H$ステップだけ$E^3$アルゴリズムを走らせ，$o\left(s, a, s^{\prime}, t\right)$のカウントを得ます．$E^3$アルゴリズムについては[Near-Optimal Reinforcement Learning in Polynomial Time](https://www.cis.upenn.edu/~mkearns/papers/KearnsSinghE3.pdf)参照．
3. 集めたカウントを利用して推定したそれぞれの遷移$\hat{P}_t$について，$\|\hat{P}_{t_1} - \hat{P}_{t_2}\|_\infty \leq \varepsilon$であるような$t_1$と$t_2$は同じ分類とします．
4. for $t=T_1 + 1, \ldots, T$ do
    1. 未知の$M_t \in \mathcal{M}$を受け取る
    2. この$M_t$が既知であるなら，...（この後がよくわからない．読みづらい...）